# Postprocessing of the UCF11 16-frame dataset

This script is used AFTER training all of the models. Once this is done, we can load all of them for inference and get all of the correctly classified test samples from the UCF11-16f dataset.

Once this is done, we can then find the intersection of ALL of the models and randomize the ordering of this to generate a randomized dataset of correctly classified test samples for all models that can be used for verification.

In [ ]:
# Configuration - Docker container paths
BASE_DIR = "/nnv/code/nnv/examples/Submission/FORMALISE2025"
DATA_DIR = f"{BASE_DIR}/data"
MODELS_DIR = f"{BASE_DIR}/models"

In [ ]:
! pip install onnxruntime

In [ ]:
import numpy as np
import os
import onnxruntime as rt

In [ ]:
dir = BASE_DIR

In [ ]:
FRAMES_PER_VIDEO = 32

# load the data
data = np.load(os.path.join(DATA_DIR, 'UCF11', 'grayscale', str(FRAMES_PER_VIDEO), f'ucf11_grayscale_{FRAMES_PER_VIDEO}f_data.npy'))
labels = np.load(os.path.join(DATA_DIR, 'UCF11', 'grayscale', str(FRAMES_PER_VIDEO), f'ucf11_grayscale_{FRAMES_PER_VIDEO}f_labels.npy'))

# create data loaders
splits = np.load(os.path.join(DATA_DIR, 'UCF11', 'grayscale', 'splits_indices.npz'))
train_idx, val_idx, test_idx = splits["train"], splits["val"], splits["test"]

test_data = data[test_idx]
test_labels = labels[test_idx]

In [ ]:
def get_correct_indices(session):
  correct_indices = []
  input_name = session.get_inputs()[0].name

  for i in range(len(test_data)):
    sample = test_data[i]
    sample = np.expand_dims(np.expand_dims(sample, 0), 0)

    output_name = session.get_outputs()[0].name
    predictions = session.run([output_name], {input_name: sample})
    predicted = np.argmax(predictions[0])

    if predicted == test_labels[i]:
      correct_indices.append(i)

  return correct_indices

In [ ]:
# load the model with 8 output channels
session = rt.InferenceSession(os.path.join(MODELS_DIR, f'ucf11_c3d_{FRAMES_PER_VIDEO}f_reducedparams_8outchannels.onnx'))
correct_indices_8out = get_correct_indices(session)
print('done with 8!')

# load the model with 16 output channels
session = rt.InferenceSession(os.path.join(MODELS_DIR, f'ucf11_c3d_{FRAMES_PER_VIDEO}f_reducedparams_16outchannels.onnx'))
correct_indices_16out = get_correct_indices(session)
print('done with 16!')

# load the model with 32 output channels
session = rt.InferenceSession(os.path.join(MODELS_DIR, f'ucf11_c3d_{FRAMES_PER_VIDEO}f_reducedparams_32outchannels.onnx'))
correct_indices_32out = get_correct_indices(session)
print('done with 32!')

# load the model with 64 output channels
session = rt.InferenceSession(os.path.join(MODELS_DIR, f'ucf11_c3d_{FRAMES_PER_VIDEO}f_reducedparams_64outchannels.onnx'))
correct_indices_64out = get_correct_indices(session)
print('done with 64!')

# load the model with 128 output channels
session = rt.InferenceSession(os.path.join(MODELS_DIR, f'ucf11_c3d_{FRAMES_PER_VIDEO}f_reducedparams_128outchannels.onnx'))
correct_indices_128out = get_correct_indices(session)
print('done with 128!')

In [ ]:
print(len(correct_indices_8out))
print(len(correct_indices_16out))
print(len(correct_indices_32out))
print(len(correct_indices_64out))
print(len(correct_indices_128out))

In [ ]:
# get the intersection of all of the correct indices to find samples
# that are correctly classified by all models
correct_indices_for_all = set(correct_indices_8out) & set(correct_indices_16out) & set(correct_indices_32out) & set(correct_indices_64out) & set(correct_indices_128out)
correct_indices_for_all = list(correct_indices_for_all)
print(len(correct_indices_for_all))
print(correct_indices_for_all)

In [ ]:
# randomize the indices to shuffle the test data randomly
import random
random.seed(42)
random.shuffle(correct_indices_for_all)

In [ ]:
correct_test_data = test_data[correct_indices_for_all]
correct_test_labels = test_labels[correct_indices_for_all]

In [ ]:
# Save verification data - must match paths expected by verifyucf11.m
np.save(os.path.join(DATA_DIR, 'UCF11', f'ucf11_grayscale_{FRAMES_PER_VIDEO}f_verification_data.npy'), correct_test_data)
np.save(os.path.join(DATA_DIR, 'UCF11', f'ucf11_grayscale_{FRAMES_PER_VIDEO}f_verification_labels.npy'), correct_test_labels)